In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Read tables from PDF
import tabula
import altair as alt



## EDA for KCSC Financial Statements v7.1a (AL Only)

In [3]:
file = '../data/KCSC Financial Statements v7.1a (AL Only).pdf'

In [4]:
# Extract the tables from the PDF
tables = tabula.read_pdf(file, pages = 'all', multiple_tables = False, pandas_options={'header': 1})

In [5]:
# Convert all the PDFs into csv
tabula.convert_into_by_batch('../data/', output_format = "csv", pages = "all")

Got stderr: Jan 08, 2021 12:00:07 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Jan 08, 2021 12:00:07 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jan 08, 2021 12:00:08 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Jan 08, 2021 12:00:08 PM org.apache.pdfbox.pdmodel.font.PDSimpleFont toUnicode



In [7]:
# Use the first table and drop the rows and columns with only NAs
df = tables[0].set_index('Unnamed: 0').dropna(how='all').T.dropna(how='all')

In [8]:
# Drop the first two rows since they only contains the number of beds and monthly rent
df = df.iloc[2:, :]

In [9]:
# Only select the comlumns with names
df = df[df.columns[~df.columns.isna()]].dropna(axis=1, how='all')

In [10]:
# Clean the data, replace the symbols
cols = [True] * len(df.columns)
for i, col in enumerate(df.columns):
    df[col] = df[col].str.replace('$', '', regex=False)
    df[col] = df[col].str.replace(',', '', regex=False)
    df[col] = df[col].str.replace('-', '', regex=False)
    df[col] = df[col].str.replace('%', '', regex=False)

In [11]:
# Replace the spaces with NAs and drop the columns with only NAs
df = df.replace(r'^\s*$', np.nan, regex=True).dropna(axis=1, how='all')

In [12]:
# Convert the data type from str to float
for col in df.columns:
    df[col] = df[col].astype('float')

In [13]:
df

Unnamed: 0,Assisted Living - Funded,Funded Bed Rent,Total Gross Rent,Funded Bed Occupancy,Gross Rent Less Vacancy,Income from Ancillary Services,Total Income,Food and Consumables,"Payroll expenses (including taxes, benefits)",Business licensing,...,Traning/development,Management Fee,Amortization (building),Amortization (furniture + equipment),Utilities,Garbage/Snow removal,Total Operating Expenses,Net Operating Income,Less: Income taxes,After tax income
JAN,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN
FEB,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN
MAR,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN
APR,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN
MAY,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN
JUN,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN
JUL,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN
AUG,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN
SEP,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN
OCT,440000.0,440000.0,440000.0,100.0,440000.0,2400.0,442400.0,22320.0,95642.0,25.0,...,800.0,24120.0,24583.0,NaN,1500.0,100.0,173515.0,268885.0,NaN,NaN


In [25]:
# Simple Bar chart for year
df_last = pd.DataFrame(df.iloc[-1,:])
df_last = df_last.reset_index()
df_last = df_last.rename(columns={'Unnamed: 0': "Categories", "Year 1": "Dollar"})
alt.Chart(df_last).mark_bar().encode(
    x = alt.Y('Categories', sort='y'),
    y = alt.Y('Dollar')
).interactive()

alt.Chart(...)

In [58]:
df_Olive = pd.read_csv("../data/Olive Devaud Proforma.csv")
df_Olive['Olive Devaud Pro-forma'] = df_Olive['Olive Devaud Pro-forma'].fillna(method='ffill')
df_Olive = df_Olive.rename(columns={'Olive Devaud Pro-forma': "Categories", "Unnamed: 1": "Items",
                        "Unnamed: 2": "Unit_Price","Unnamed: 3": "Amount",
                        "Unnamed: 4": "Units","Unnamed: 5": "Sub_Total",
                        "Unnamed: 6": "Unknown","Unnamed: 7": "Grand_total",
                        "Unnamed: 8": "BUDGET"})
df_Olive = df_Olive.iloc[1:]

In [65]:
print(df_Olive['Categories'].unique().tolist())

['PROJECT OVERVIEW', 'ADDRESS', 'PROJECT INFORMATION', 'RENTAL INCOME PER ANNUM', 'GRAND TOTAL', 'DEVELOPMENT BUDGET', 'Land', 'Soft Cost', 'Municipal', 'Consultants', 'Insurance', 'Construction', 'Finance', 'Lease Marketing', 'Project Contingency', 'Project Management Fee', 'PROFITABILITY ANALYSIS - BASED ON RENTAL INCOME PER ANNUM', 'Construction Mortgage', 'Net Value', 'Total Cost', 'Profit on Project', 'Return on Cost', 'Rental Properties Market Value', 'Rental Income Per Annum']


In [71]:
#Create sub_data frame for each categoty
for i, g in df_Olive.groupby('Categories'):
    globals()['df_' + str(i).replace(" ", "")] =  g

In [73]:
df_RENTALINCOMEPERANNUM

,Categories,Items,Unit_Price,Amount,Units,Sub_Total,Unknown,Grand_total,BUDGET
39,RENTAL INCOME PER ANNUM,Asset Value,NaN,NaN,NaN,NaN,NaN,NaN,"$ 24,425,633"
40,RENTAL INCOME PER ANNUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,RENTAL INCOME PER ANNUM,_Assisted Living (Lease),NaN,"16,400",sqft,"$ 2,821,500.00",NaN,"$ 1 ,410,750.00",NaN
42,RENTAL INCOME PER ANNUM,NaN,NaN,55,units,NaN,NaN,NaN,NaN
43,RENTAL INCOME PER ANNUM,NaN,NaN,95%,Occupancy,NaN,NaN,NaN,NaN
44,RENTAL INCOME PER ANNUM,NaN,NaN,"$4,500",per month,NaN,NaN,NaN,NaN
45,RENTAL INCOME PER ANNUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
46,RENTAL INCOME PER ANNUM,_Independent Living (Lease),NaN,"16,400",sqft,"$ 1,037,400.00",NaN,"$ 518,700.00",NaN
47,RENTAL INCOME PER ANNUM,NaN,NaN,35,units,NaN,NaN,NaN,NaN
48,RENTAL INCOME PER ANNUM,NaN,NaN,95%,Occupancy,NaN,NaN,NaN,NaN
